**Importando bibliotecas**

In [354]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


**Importando bases de dados**

In [371]:
df_geocode = pd.read_csv('./bases_finais/final_df_geocode.csv', sep=';')
test_data = pd.read_csv('./bases_finais/final_test_data.csv', sep=';') # Testing Cost = custo de aplicação, custo para o laboratório
transactional_data = pd.read_csv('./bases_finais/final_transactional_data.csv', sep=';') # Testing Cost = Custo para o paciente
EconomicData_ZCTAs = pd.read_csv('./bases_finais/final_EconomicData_ZCTAs.csv', sep=';') 
DemographicData_ZCTAs = pd.read_csv('./bases_finais/final_DemographicData_ZCTAs.csv', sep=';')


**Storytelling**

Principais dados a serem analisados:
- População total do Zipcode
- Poder aquisitivo médio do Zipcode
- Média de idade do Zipcode
- Proporção de homens para mulheres do Zipcode

**1) População total do Zipcode**

Hipótese: Quanto maior a população local do Zipcode, maior será a quantidade do público e clientes do laboratório.

In [356]:
# separando laboratórios mais frequentados por quartis

transactional_data.lab_id.value_counts().quantile([0.25,0.5,0.75])

0.25     5466.25
0.50    12186.50
0.75    22911.25
Name: lab_id, dtype: float64

In [357]:
# pegando dados do 1° quartil
transactional_data_25 = transactional_data['lab_id'].value_counts()
transactional_data_25 = transactional_data_25.to_frame()
transactional_data_25 = transactional_data_25[transactional_data_25.lab_id <= 5466.25]
transactional_data_25.reset_index(inplace=True)
transactional_data_25 = transactional_data_25.rename(columns={'index': 'lab_id', 'lab_id':'contagem'})

# pegando dados do 2° quartil
transactional_data_50 = transactional_data['lab_id'].value_counts()
transactional_data_50 = transactional_data_50.to_frame()
transactional_data_50 = transactional_data_50[transactional_data_50.lab_id <= 12186.50]
transactional_data_50 = transactional_data_50[transactional_data_50.lab_id > 5466.25]
transactional_data_50.reset_index(inplace=True)
transactional_data_50 = transactional_data_50.rename(columns={'index': 'lab_id', 'lab_id':'contagem'})

# pegando dados do 3° quartil
transactional_data_75 = transactional_data['lab_id'].value_counts()
transactional_data_75 = transactional_data_75.to_frame()
transactional_data_75 = transactional_data_75[transactional_data_75.lab_id > 12186.50]
transactional_data_75.reset_index(inplace=True)
transactional_data_75 = transactional_data_75.rename(columns={'index': 'lab_id', 'lab_id':'contagem'})


In [360]:
# dando merge nos laboratorios do 1° quartil com os dados do df_geocode
df_geocode_combinando_transactional_data_25 = df_geocode.copy()
df_geocode_combinando_transactional_data_25.zipcode = df_geocode_combinando_transactional_data_25.zipcode.astype('object')
df_geocode_combinando_transactional_data_25 = df_geocode_combinando_transactional_data_25.merge(transactional_data_25, how="inner", on="lab_id")
df_geocode_combinando_transactional_data_25 = df_geocode_combinando_transactional_data_25.drop(columns=['address', 'location'])

# DemographicData_ZCTAs.zipcode = DemographicData_ZCTAs.zipcode.str.slice(start=6)
df_geocode_combinando_transactional_data_25_e_DemographicData_ZCTAs = df_geocode_combinando_transactional_data_25.copy()
df_geocode_combinando_transactional_data_25_e_DemographicData_ZCTAs
df_geocode_combinando_transactional_data_75_e_DemographicData_ZCTAs = df_geocode_combinando_transactional_data_75.copy()
df_geocode_combinando_transactional_data_75_e_DemographicData_ZCTAs
DemographicData_ZCTAs.zipcode = DemographicData_ZCTAs.zipcode.astype('float')

# dando merge no dataframe que combina os dados dos laboratorios do df_geocode e do 1° quartil com as informações dos zipcodes dos laboratórios
df_geocode_combinando_transactional_data_25_e_DemographicData_ZCTAs = df_geocode_combinando_transactional_data_25_e_DemographicData_ZCTAs.merge(DemographicData_ZCTAs, how="inner", on="zipcode")

# deletando colunas desnecessárias
df_geocode_combinando_transactional_data_25_e_DemographicData_ZCTAs = df_geocode_combinando_transactional_data_25_e_DemographicData_ZCTAs.drop(columns=['id', 'SexRatio(males per 100 females)', 'Population_Under5Years', 'Population_5to9Years', 'Population_10to14Years', 'Population_15to19Years','Population_20to24Years', 'Population_25to34Years', 'Population_35to44Years', 'Population_45to54Years','Population_55to59Years','Population_60to64Years','Population_65to74Years','Population_75to84Years','Population_85YearsAndOver', 'MedianAgeInYears'])
df_geocode_combinando_transactional_data_25_e_DemographicData_ZCTAs

,lab_id,zipcode,contagem,total_population
0,L545,61469,2805,2213
1,L518,94566,1916,45893
2,L504,61282,3164,7732
3,L070,19036,5066,12894
4,L720,98034,1835,43471
5,L889,19096,5250,13695
6,L529,2762,3858,9183
7,L530,19004,3800,9524
8,L519,13057,5446,13643
9,L067,95204,3901,28667


In [365]:
# média populacional dos laboratórios do 1° quartil
media = df_geocode_combinando_transactional_data_25_e_DemographicData_ZCTAs.total_population.mean()

print(f'A média da população dos zipcodes do 1° quartil é de {media:.1f} habitantes')

A média da população dos zipcodes do 1° quartil é de 19125.7 habitantes


In [361]:
# dando merge nos laboratorios do 3° quartil com os dados do df_geocode
df_geocode_combinando_transactional_data_75 = df_geocode.copy()
df_geocode_combinando_transactional_data_75.zipcode = df_geocode_combinando_transactional_data_75.zipcode.astype('object')
df_geocode_combinando_transactional_data_75 = df_geocode_combinando_transactional_data_75.merge(transactional_data_75, how="inner", on="lab_id")
df_geocode_combinando_transactional_data_75 = df_geocode_combinando_transactional_data_75.drop(columns=['address', 'location'])

# dando merge no dataframe que combina os dados dos laboratorios do df_geocode e do 3° quartil com as informações dos zipcodes dos laboratórios
df_geocode_combinando_transactional_data_75_e_DemographicData_ZCTAs = df_geocode_combinando_transactional_data_75_e_DemographicData_ZCTAs.merge(DemographicData_ZCTAs, how="inner", on="zipcode")
df_geocode_combinando_transactional_data_75_e_DemographicData_ZCTAs



,lab_id,zipcode,contagem,id,total_population,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears
0,L152,30096,95256,8600000US30096,68691,89.0,4335,4376,4749,4130,4874,11188,9629,10248,4386,3492,4636,1988,660,35.8
1,L520,13440,20481,8600000US13440,41014,104.2,2673,2497,2254,1832,2360,6157,4432,4841,3431,3168,3819,2456,1094,40.5
2,L141,98034,50631,8600000US98034,43471,98.8,2813,2485,2426,1846,2392,8207,6852,5702,2235,2712,3682,1496,623,36.8
3,L140,98034,23726,8600000US98034,43471,98.8,2813,2485,2426,1846,2392,8207,6852,5702,2235,2712,3682,1496,623,36.8
4,L524,30342,13187,8600000US30342,30873,94.0,1963,2322,2150,1701,1638,6419,4496,4319,1820,1257,1854,633,301,33.4
5,L172,30281,20259,8600000US30281,68456,92.7,4603,4650,5288,5023,4807,9170,9202,9349,4997,3739,4844,2131,653,35.7
6,L531,94588,17707,8600000US94588,37253,91.5,1757,2596,3377,2460,1453,3768,5837,6159,2310,2389,2859,1764,524,40.7
7,L160,1840,33938,8600000US01840,5573,96.4,437,406,380,282,474,775,717,662,383,299,463,175,120,35.2
8,L533,94598,12826,8600000US94598,27831,89.4,1871,1660,1459,1564,877,2782,3674,3808,1841,1953,3403,1906,1033,45.1
9,L061,98004,46043,8600000US98004,36202,105.3,1692,1450,1958,1343,1836,8340,5246,5095,2201,1753,2597,1762,929,37.8


In [366]:
# média populacional dos laboratórios do 3° quartil
media = df_geocode_combinando_transactional_data_75_e_DemographicData_ZCTAs.total_population.mean()

print(f'A média da população dos zipcodes do 3° quartil é de {media:.1f} habitantes')

A média da população dos zipcodes do 3° quartil é de 44791.9 habitantes


**Conclusão:** Como a média da população local dos zipcodes do 3° quartil é muito maior do que a média dos zipcodes do 1° quartil, e considerando que esses quartis foram divididos de acordo com a quantidade de serviços feitos em cada laboratório, ou seja, os laboratórios do 3° quartil realizaram mais serviços do que os do 1° quartil, então é possível dizer que a população total do zipcode afeta diretamente na quantidade de serviços realizados pelo laboratório, e que para a análise de futuros laboratórios da rede, seria interessante observar zipcodes com uma população maior. 

*Média da população dos zipcodes*

In [ ]:
media = DemographicData_ZCTAs.total_population.mean()
print(f'A média da população dos zipcodes analisados é de {media:.0f} habitantes')

*Zipcodes com a maior população*

In [ ]:
max_population_DemographicData_ZCTAs = DemographicData_ZCTAs.copy()
max_population_DemographicData_ZCTAs.sort_values(by=['total_population'], ascending=False, inplace=True)
max_population_DemographicData_ZCTAs[['total_population']].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])

Como possíveis destinos para futuros laboratórios da rede, seria interessante que considerássemos locais apenas com a população total entre os 10% dos dados analisados, ou seja, zipcodes com a população acima de 31391 habitantes. 

In [ ]:
top_10_DemographicData_ZCTAs = max_population_DemographicData_ZCTAs[max_population_DemographicData_ZCTAs['total_population'] >= 31391]
top_10_DemographicData_ZCTAs.reset_index(inplace=True, drop=True)
top_10_DemographicData_ZCTAs

-------------------------------------------------------------------------------------------------------------------------------

**2) Poder aquisitivo médio do Zipcode**

Hipótese: Quanto maior for o poder aquisitivo médio da população do zipcode, maior seria o público disposto a utilizar os serviços do laboratório.

In [367]:
EconomicData_ZCTAs.columns

Index(['id', 'geographic_area_name', 'TotalHouseholds_LessThan$10.000',
       'TotalHouseholds_$10.000to$14.999', 'TotalHouseholds_$15.000to$24.999',
       'TotalHouseholds_$25.000to$34.999', 'TotalHouseholds_$35.000to$49.999',
       'TotalHouseholds_$50.000to$74.999', 'TotalHouseholds_$75.000to$99.999',
       'TotalHouseholds_$100.000to$149.999',
       'TotalHouseholds_$150.000to$199.999', 'TotalHouseholds_$200.000OrMore'],
      dtype='object')

In [373]:
purchasing_power_EconomicData_ZCTAs = EconomicData_ZCTAs.copy()
purchasing_power_EconomicData_ZCTAs['average'] = (((purchasing_power_EconomicData_ZCTAs['TotalHouseholds_LessThan$10.000'] * 10000) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$10.000to$14.999'] * 12250) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$15.000to$24.999'] * 20000) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$25.000to$34.999'] * 30000) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$35.000to$49.999'] * 42250) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$50.000to$74.999'] * 62250) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$75.000to$99.999'] * 87250) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$100.000to$149.999'] * 125000) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$150.000to$199.999'] * 175000) +
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$200.000OrMore'] * 200000))/
                                                  (purchasing_power_EconomicData_ZCTAs['TotalHouseholds_LessThan$10.000'] + 
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$10.000to$14.999'] +
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$15.000to$24.999'] +
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$25.000to$34.999'] +
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$35.000to$49.999'] + 
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$50.000to$74.999'] + 
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$75.000to$99.999'] + 
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$100.000to$149.999'] + 
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$150.000to$199.999'] + 
                                                   purchasing_power_EconomicData_ZCTAs['TotalHouseholds_$200.000OrMore'])).round(0)

In [376]:

# purchasing_power_EconomicData_ZCTAs.zipcode = purchasing_power_EconomicData_ZCTAs.zipcode.str.slice(start=6)
df_geocode_combinando_transactional_data_25_e_purchasing_power_EconomicData_ZCTAs = df_geocode_combinando_transactional_data_25.copy()
df_geocode_combinando_transactional_data_25_e_purchasing_power_EconomicData_ZCTAs
df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs = df_geocode_combinando_transactional_data_75.copy()
df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs
purchasing_power_EconomicData_ZCTAs.zipcode = purchasing_power_EconomicData_ZCTAs.zipcode.astype('float')

# dando merge no dataframe que combina os dados dos laboratorios do df_geocode e do 1° quartil com as informações dos zipcodes dos laboratórios
df_geocode_combinando_transactional_data_25_e_purchasing_power_EconomicData_ZCTAs = df_geocode_combinando_transactional_data_25_e_purchasing_power_EconomicData_ZCTAs.merge(purchasing_power_EconomicData_ZCTAs, how="inner", on="zipcode")
df_geocode_combinando_transactional_data_25_e_purchasing_power_EconomicData_ZCTAs

# dando merge no dataframe que combina os dados dos laboratorios do df_geocode e do 3° quartil com as informações dos zipcodes dos laboratórios
df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs = df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs.merge(purchasing_power_EconomicData_ZCTAs, how="inner", on="zipcode")
df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs

# deletando colunas desnecessárias
# df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs = df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs.drop(columns=['id', 'SexRatio(males per 100 females)', 'Population_Under5Years', 'Population_5to9Years', 'Population_10to14Years', 'Population_15to19Years','Population_20to24Years', 'Population_25to34Years', 'Population_35to44Years', 'Population_45to54Years','Population_55to59Years','Population_60to64Years','Population_65to74Years','Population_75to84Years','Population_85YearsAndOver', 'MedianAgeInYears'])
# df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs

,lab_id,zipcode,contagem,id,TotalHouseholds_LessThan$10.000,TotalHouseholds_$10.000to$14.999,TotalHouseholds_$15.000to$24.999,TotalHouseholds_$25.000to$34.999,TotalHouseholds_$35.000to$49.999,TotalHouseholds_$50.000to$74.999,TotalHouseholds_$75.000to$99.999,TotalHouseholds_$100.000to$149.999,TotalHouseholds_$150.000to$199.999,TotalHouseholds_$200.000OrMore,average
0,L545,61469,2805,8600000US61469,16,72,133,108,175,287,112,107,10,0,55979.0
1,L518,94566,1916,8600000US94566,262,357,792,576,687,1245,1438,2317,2504,6297,137137.0
2,L504,61282,3164,8600000US61282,453,210,216,342,339,773,516,485,97,39,61829.0
3,L070,19036,5066,8600000US19036,190,241,588,714,503,904,654,975,344,214,74619.0
4,L720,98034,1835,8600000US98034,672,305,629,760,1474,2226,2609,3797,2317,2857,109136.0
5,L889,19096,5250,8600000US19096,158,72,287,256,347,670,518,807,673,1854,125292.0
6,L529,2762,3858,8600000US02762,103,15,265,244,195,556,613,809,419,447,102023.0
7,L530,19004,3800,8600000US19004,182,100,180,227,254,408,326,619,604,1001,118252.0
8,L519,13057,5446,8600000US13057,218,370,554,565,722,1265,805,1075,304,243,73583.0
9,L067,95204,3901,8600000US95204,678,785,1085,1252,1401,2136,1415,1425,363,536,66822.0


In [377]:
media = df_geocode_combinando_transactional_data_25_e_purchasing_power_EconomicData_ZCTAs.average.mean()

print(f'A média dos salários dos zipcodes do 1° quartil é de {media:.1f} dólares')

A média dos salários dos zipcodes do 1° quartil é de 88276.7 dólares


In [378]:
media = df_geocode_combinando_transactional_data_75_e_purchasing_power_EconomicData_ZCTAs.average.mean()

print(f'A média dos salários dos zipcodes do 3° quartil é de {media:.1f} dólares')

A média dos salários dos zipcodes do 3° quartil é de 88194.2 dólares


**Conclusão:** A média dos salários dos zipcodes do 1° quartil é quase igual e até maior do que as dos zipcodes do 3° quartil, rejeitando a hipótese de que o poder aquisitivo interfira fortemente na utilização dos laboratórios. 

-------------------------------------------------------------------------------------------------------------------------------

**3) Média de idade do Zipcode**

Hipótese: Pessoas mais velhas são mais propensas a utilizar os serviços dos laboratórios.

In [ ]:
pacientes_unicos_transactional_data = transactional_data.copy()
pacientes_unicos_transactional_data = pacientes_unicos_transactional_data.drop_duplicates(subset=['patient_id'], keep='first') # pacientes sem repetição
pacientes_unicos_transactional_data.reset_index(inplace=True, drop=True) # resetando index
pacientes_unicos_transactional_data['date_of_service'] = pd.to_datetime(pacientes_unicos_transactional_data['date_of_service'], format='%Y-%m-%d')
pacientes_unicos_transactional_data['date_of_birth'] = pd.to_datetime(pacientes_unicos_transactional_data['date_of_birth'], format='%Y-%m-%d %H:%M:%S') # cortando a hora das datas de nascimento
pacientes_unicos_transactional_data['year'] = pd.DatetimeIndex(pacientes_unicos_transactional_data['date_of_birth']).year
pacientes_unicos_transactional_data

In [ ]:
pacientes_unicos_transactional_data['year'] = pd.DatetimeIndex(pacientes_unicos_transactional_data['date_of_birth']).year
pacientes_unicos_transactional_data

In [ ]:
# calculando média de idade dos pacientes
media = (2022 - (pacientes_unicos_transactional_data.year.mean()))
print(f'A média de idade de pessoas que frequentam os laboratórios é {media:.1f}')
pacientes_unicos_transactional_data.drop(columns=['year'], inplace = True)

**Conclusão:** É possível concluir então que pessoas por volta dos 40 anos são as que mais frequentam os laboratórios da rede, assim, seria interessante procurarmos por zipcodes que tenham uma média de idade próxima a esse valor

-------------------------------------------------------------------------------------------------------------------------------

**4) Proporção de homens para mulheres do Zipcode**

Hipótese: Pessoas do gênero feminino são mais propensas a utilizar os serviços dos laboratórios.

In [ ]:
pacientes_unicos_transactional_data.gender.value_counts()

In [ ]:
proporcao = 341863/(pacientes_unicos_transactional_data.gender.count())
print(f'{(proporcao * 100):.1f}% dos pacientes são do gênero feminino')

**Conclusão:** É possível concluir então que pessoas do gênero feminino possuem maior tendência de frequentar o laboratório, por isso, seria interessante procurarmos locais que tenham uma alta proporção de pessoas do gênero feminino em relação ao masculino.

-------------------------------------------------------------------------------------------------------------------------------

In [ ]:
transactional_data.cod_item.value_counts()